In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import math
import random
import matplotlib.pyplot as plt
import shutil
from sklearn.preprocessing import QuantileTransformer
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import pandas as pd

# List of CSV files for training
csv_files_driving = ["/kaggle/input/car-hacking-challenge-dataset/0_Preliminary/0_Training/Pre_train_D_0.csv", "/kaggle/input/car-hacking-challenge-dataset/0_Preliminary/0_Training/Pre_train_D_1.csv", "/kaggle/input/car-hacking-challenge-dataset/0_Preliminary/0_Training/Pre_train_D_2.csv"]
csv_files_stationary = ["/kaggle/input/car-hacking-challenge-dataset/0_Preliminary/0_Training/Pre_train_S_0.csv", "/kaggle/input/car-hacking-challenge-dataset/0_Preliminary/0_Training/Pre_train_S_1.csv", "/kaggle/input/car-hacking-challenge-dataset/0_Preliminary/0_Training/Pre_train_S_2.csv"]

# Combine all CSV files into a single DataFrame
combined_df = pd.DataFrame()

for file in csv_files_driving + csv_files_stationary:
    df = pd.read_csv(file)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv("/kaggle/working/Training_Final.csv", index=False)

print("Files combined successfully into training.csv")


In [ ]:
#Read dataset
df=pd.read_csv('/kaggle/working/Training_Final.csv')

In [ ]:
df.Class.value_counts()

In [ ]:
df.SubClass.value_counts()

# Data Transformation
Convert tabular data to images Procedures:

Use quantile transform to transform the original data samples into the scale of [0,255], representing pixel value

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import QuantileTransformer
import matplotlib.pyplot as plt
import os

def hex_to_int(hex_str):
    """Convert a hexadecimal string to an integer."""
    if isinstance(hex_str, str):
        return int(hex_str, 16)
    return hex_str  # If it's not a string, return the original value

def preprocess_data(df):
    """Convert hexadecimal fields to integers and normalize the data."""
    # Convert Arbitration_ID to integer
    df['Arbitration_ID'] = df['Arbitration_ID'].apply(hex_to_int)

    # Convert Data field to integer and split into separate columns
    data_columns = df['Data'].str.split(' ', expand=True)

    #print(type(data_columns))

    for i in range(8):
        data_columns[i] = data_columns[i].apply(hex_to_int)
        data_columns.rename(columns={i: f'Data[{i}]'}, inplace=True)


    #print(data_columns)

    df = df.drop(columns=['Data'])
    df = pd.concat([df, data_columns], axis=1)

    return df

# Load the combined CSV file
df = pd.read_csv('/kaggle/working/Training_Final.csv')

# Preprocess the data
df_changed = preprocess_data(df)

# Sample 20% of the dataset
df_changed = df_changed.sample(frac=0.2, random_state=42)

# Reset index to keep it clean
df_changed = df_changed.reset_index(drop=True)

# Display the resulting DataFrame
df_changed




In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
data = df_changed

# Handle missing values (fill NaNs with a method or drop rows)
data.fillna(method='ffill', inplace=True)  # Example: forward fill

# Convert 'Class' to binary values (0 for Normal, 1 for Other)
data['Class'] = data['Class'].apply(lambda x: 0 if x == 'Normal' else 1)

# Encode 'Subclass' for abnormal data (it will be NaN for normal data)
label_encoder = LabelEncoder()
data['SubClass'] = data['SubClass'].fillna('None')  # Fill NaNs with 'None'
data['SubClass'] = label_encoder.fit_transform(data['SubClass'])

# Define features and target variable
features = 


X = data[features]
y = data['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model (Random Forest in this case)
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

# Save the model if needed
import joblib
joblib.dump(model, 'model.pkl')


In [ ]:
import pandas as pd
import numpy as np
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras import layers, models

# Load the dataset
data = df_changed

# Handle missing values (e.g., fill NaNs with a method or drop rows)
data.fillna(method='ffill', inplace=True)

# Encode 'Class' to binary values (0 for Normal, 1 for Attack)
data['Class'] = data['Class'].apply(lambda x: 0 if x == 'Normal' else 1)

# Encode 'Subclass' (if needed)
data['SubClass'] = data['SubClass'].fillna('None')
label_encoder = LabelEncoder()
data['SubClass'] = label_encoder.fit_transform(data['SubClass'])

# Define features and target variable
features = ['Arbitration_ID', 'DLC', 'Data[0]', 'Data[1]', 'Data[2]', 'Data[3]', 'Data[4]', 'Data[5]', 'Data[6]', 'Data[7]', 'SubClass']
X = data[features]
y = data['Class']

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

# Save the model if needed
model.save('model.h5')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.utils import to_categorical

# Load and preprocess data
data = df_changed 

# Handle missing values
data.fillna(method='ffill', inplace=True)

# Encode 'Class' to binary values (0 for Normal, 1 for Attack)
data['Class'] = data['Class'].apply(lambda x: 0 if x == 'Normal' else 1)

# Encode 'SubClass'
data['SubClass'] = data['SubClass'].fillna('None')
label_encoder = LabelEncoder()
data['SubClass'] = label_encoder.fit_transform(data['SubClass'])

# Define features and target variable
features = ['Arbitration_ID', 'DLC', 'Data[0]', 'Data[1]', 'Data[2]', 'Data[3]', 'Data[4]', 'Data[5]', 'Data[6]', 'Data[7]', 'SubClass']
X = data[features]
y = data['Class']

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for CNN (e.g., to 3x3x1 if we want to fit it into a grid)
X = X.reshape(X.shape[0], 11, 1, 1)

# Convert labels to categorical (if necessary)
y = to_categorical(y, num_classes=2)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = models.Sequential()

# First Convolutional Block
model.add(layers.Conv2D(64, (3, 3), strides=(1, 1), input_shape=(11, 1, 1), padding='same', activation='relu'))
model.add(layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(1, 1)))

# Second Convolutional Block
model.add(layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(1, 1)))

# Third Convolutional Block
model.add(layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(layers.GlobalAveragePooling2D())

# Fully Connected Layers
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_accuracy', patience=2, verbose=1, mode='auto')
save_best_model = callbacks.ModelCheckpoint(filepath='model.weights.h5', monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto')

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping, save_best_model])

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

# Plot training history
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()



## VGG16 Model Training

In [ ]:
import pandas as pd
import numpy as np
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# Load and preprocess data
data = df_changed 

# Handle missing values
data.fillna(method='ffill', inplace=True)

# Encode 'Class' to binary values (0 for Normal, 1 for Attack)
data['Class'] = data['Class'].apply(lambda x: 0 if x == 'Normal' else 1)

# Encode 'SubClass'
data['SubClass'] = data['SubClass'].fillna('None')
label_encoder = LabelEncoder()
data['SubClass'] = label_encoder.fit_transform(data['SubClass'])

# Define features and target variable
features = ['Arbitration_ID', 'DLC', 'Data[0]', 'Data[1]', 'Data[2]', 'Data[3]', 'Data[4]', 'Data[5]', 'Data[6]', 'Data[7]', 'SubClass']
X = data[features]
y = data['Class']

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape data for CNN (e.g., to 3x3x1 if we want to fit it into a grid)
X = X.reshape(X.shape[0], 11, 1, 1)

# Convert labels to categorical (if necessary)
y = to_categorical(y, num_classes=2)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# Build the VGG16 model compatible with your data

model_vgg16 = models.Sequential()

# Block 1
model_vgg16.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(11, 1, 1)))
model_vgg16.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model_vgg16.add(layers.MaxPooling2D(pool_size=(1, 1)))
model_vgg16.add(layers.BatchNormalization())

# Block 2
model_vgg16.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model_vgg16.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model_vgg16.add(layers.MaxPooling2D(pool_size=(1, 1)))
model_vgg16.add(layers.BatchNormalization())

# Block 3
model_vgg16.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model_vgg16.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model_vgg16.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model_vgg16.add(layers.MaxPooling2D(pool_size=(1, 1)))
model_vgg16.add(layers.BatchNormalization())

# Block 4
model_vgg16.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
model_vgg16.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
model_vgg16.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
model_vgg16.add(layers.MaxPooling2D(pool_size=(1, 1)))
model_vgg16.add(layers.BatchNormalization())

# Block 5
model_vgg16.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
model_vgg16.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
model_vgg16.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
model_vgg16.add(layers.MaxPooling2D(pool_size=(1, 1)))
model_vgg16.add(layers.BatchNormalization())

# Flatten and Fully Connected Layers
model_vgg16.add(layers.Flatten())
model_vgg16.add(layers.Dense(4096, activation='relu'))
model_vgg16.add(layers.Dropout(0.5))
model_vgg16.add(layers.Dense(4096, activation='relu'))
model_vgg16.add(layers.Dropout(0.5))
model_vgg16.add(layers.Dense(2, activation='softmax'))  # Output layer for binary classification

optimizer = Adam(learning_rate=0.0001)
# Compile the model
model_vgg16.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_accuracy', patience=2, verbose=1, mode='auto')
save_best_model = callbacks.ModelCheckpoint(filepath='vgg_16model.weights.h5', monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto')

# Train the model
history_vgg16 = model_vgg16.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping, save_best_model])

# Evaluate the model
test_loss_vgg16, test_acc_vgg16 = model_vgg16.evaluate(X_test, y_test)
print(f"Test Accuracy VGG16: {test_acc_vgg16}")

# Plot training history
plt.plot(history_vgg16.history['accuracy'], label='accuracy')
plt.plot(history_vgg16.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Function to compute entropy
def get_H(series_aid):
    count = series_aid.value_counts()
    p_i = count / series_aid.shape[0]
    return -(p_i * np.log(p_i)).sum()

# Dataset files
files = ['Pre_train_D_0.csv', 'Pre_train_S_2.csv', 'Pre_train_D_2.csv', 
         'Pre_train_S_0.csv', 'Pre_train_S_1.csv', 'Pre_train_D_1.csv']
dataset_path = "/kaggle/input/car-hacking-challenge-dataset/0_Preliminary/0_Training"

# **Step 1: Gather all unique SubClass values**
subclass_values = set()  # Using a set to avoid duplicates
df_list = []

# Load dataset function
for file in files:
    df = pd.read_csv(f"{dataset_path}/{file}")

    print(f"Processing: {file} | Columns: {df.columns.tolist()}")

    # If 'SubClass' exists, collect its unique values
    if 'SubClass' in df.columns:
        subclass_values.update(df['SubClass'].dropna().unique())
    else:
        df['SubClass'] = "Unknown"  # Assign 'Unknown' where SubClass is missing

    df_list.append(df)

# **Step 2: Include 'Unknown' in LabelEncoder training**
subclass_values.add("Unknown")  # Ensure 'Unknown' is part of the encoding
label_encoder = LabelEncoder()
label_encoder.fit(list(subclass_values))  # Convert set to list and fit

# **Step 3: Process each dataset and apply the same label encoding**
processed_dfs = []
for df in df_list:
    # Fill missing 'SubClass' with 'Unknown' before encoding
    df['SubClass'] = df['SubClass'].fillna("Unknown")

    # Convert timestamp features
    df['abstime'] = pd.to_datetime(df['Timestamp'], unit='s').round('us')
    df['monotime'] = df['Timestamp'] - df['Timestamp'].min()

    # Convert Arbitration_ID from hex to int
    df['aid_int'] = df['Arbitration_ID'].map(lambda x: int(x, 16))

    # Encode Class (Normal=0, Attack=1)
    df['y'] = df['Class'].map({'Normal': 0, 'Attack': 1})

    # Compute time interval per Arbitration_ID
    df['time_interval'] = df.groupby('Arbitration_ID')['Timestamp'].diff()

    # Convert Data field from hex to numerical
    df['Data'] = df['Data'].fillna('')  # Fill missing Data with empty string
    df_data = [[int(i, 16) for i in row.split()]+[-1]*(8-len(row.split())) for row in df['Data']]
    df_data = pd.DataFrame(df_data, columns=[f'datafield{i}' for i in range(8)])

    # Apply the pre-trained LabelEncoder to 'SubClass'
    df['SubClass'] = label_encoder.transform(df['SubClass'])

    # Merge processed data
    df_final = pd.concat([df.reset_index(drop=True), df_data.reset_index(drop=True)], axis=1)
    processed_dfs.append(df_final)

# Combine datasets
df_combined = pd.concat(processed_dfs, ignore_index=True)

# Compute entropy function
df_combined['entropy'] = df_combined.rolling(window=2402, min_periods=2402, step=10)['aid_int'].apply(get_H)
df_combined['entropy'] = df_combined['entropy'].ffill()

# Remove rows with NaN time_interval or entropy
df_combined = df_combined.dropna(subset=['time_interval', 'entropy'])

# Select features and target
features = ['aid_int', 'time_interval', 'entropy', 'datafield0', 'datafield1', 
            'datafield2', 'datafield3', 'datafield4', 'datafield5', 'datafield6', 
            'datafield7', 'SubClass']
X = df_combined[features]
y = df_combined['y']

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape X for CNN or LSTM models
X = X.reshape(X.shape[0], X.shape[1], 1)

# Save processed data
df_combined.to_csv("processed_dataset.csv", index=False)

print(f"✅ Preprocessing complete! Processed data shape: {X.shape}")

# Print the mapping of SubClass labels
mapping = {index: label for index, label in enumerate(label_encoder.classes_)}
print("📌 SubClass Mapping:", mapping)


Processing: Pre_train_D_0.csv | Columns: ['Timestamp', 'Arbitration_ID', 'DLC', 'Data', 'Class']
Processing: Pre_train_S_2.csv | Columns: ['Timestamp', 'Arbitration_ID', 'DLC', 'Data', 'Class', 'SubClass']
Processing: Pre_train_D_2.csv | Columns: ['Timestamp', 'Arbitration_ID', 'DLC', 'Data', 'Class', 'SubClass']
Processing: Pre_train_S_0.csv | Columns: ['Timestamp', 'Arbitration_ID', 'DLC', 'Data', 'Class']
Processing: Pre_train_S_1.csv | Columns: ['Timestamp', 'Arbitration_ID', 'DLC', 'Data', 'Class', 'SubClass']
Processing: Pre_train_D_1.csv | Columns: ['Timestamp', 'Arbitration_ID', 'DLC', 'Data', 'Class', 'SubClass']
✅ Preprocessing complete! Processed data shape: (3669343, 12, 1)
📌 SubClass Mapping: {0: 'Flooding', 1: 'Fuzzing', 2: 'Normal', 3: 'Replay', 4: 'Spoofing', 5: 'Unknown'}


In [ ]:
import pandas as pd

# Path to the Kaggle output directory
input_path = "/kaggle/working/large_file.xlsx"  # Adjust if needed
output_path = "/kaggle/working/output.csv"

# Load the Excel file
df = pd.read_excel(input_path, engine="openpyxl")

# Save as CSV
df.to_csv(output_path, index=False)

print("Conversion complete! CSV saved at:", output_path)
